In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import json
from pathlib import Path

pd.options.plotting.backend = "plotly"

from ares_py.get_ld import get_ld
from ares_py.mcs.mcs_header import mcs_get_header_version
from ares_py.mcs.mcs_read import mcs_read_data
from ares_py.geometry.array_geom import (
    calc_geom_factor,
    load_doi_data,
    get_2d_geom_factor,
)


ld = get_ld("mcs")
ld = ld.loc[ld["fp"].str.contains("A2_Dip-Dip overlap HD")]

for fp in ld["fp"]:
    with open(fp, "r", encoding="utf-8") as file:
        lines = file.readlines()
        lines = pd.Series(lines)
        header_version, header_data = mcs_get_header_version(lines)
        data = mcs_read_data(fp, header_data, header_version)
# data[10]

gf2d = get_2d_geom_factor(data[10])

df = data[10]

In [161]:
el_1d = data[10].iloc[:, :13].astype(float)
el_1d = np.array(el_1d)


def mcs_1d_to_2d(array, shape):
    array = array.reshape(-1, 1)
    array = np.repeat(array, shape[1], axis=1)
    return array


rows, cols = el_1d.shape
el_2d = np.full(shape=(rows, cols, cols), fill_value=np.nan)

for i in range(cols):
    el_2d[:, :, i] = mcs_1d_to_2d(el_1d[:, i], (rows, cols, cols))

el_2d = el_2d[:, 2:, :]

l = np.nanmax(el_2d, axis=2) - np.nanmin(el_2d, axis=2)

In [166]:
# Wenner, a = l/3, n=1, k= x*2pi*a (alfa x=1, beta x=2, gamma x=3)
wn_a = l / 3
wn_n = np.full((rows, cols), 1)
wna_k = 2 * np.pi * wn_a
wnb_k = 6 * np.pi * wn_a
wnc_k = 3 * np.pi * wn_a
# pole pole - a= p1-c1, n =1, k=2pi*a
pp_n = np.full((rows, cols), 1)
pp_a = el_1d[:, 2:] - el_2d[:, :, 0]
pp_k = 2 * np.pi * pp_a

# DD a=c1-c2, n =? , k=pi*n*(n+1)*(n+2)*a
dd_a = el_2d[:, :, 0] - el_2d[:, :, 1]
# dd_n = (el_1d[:, 2:] - el_2d[:, :, 1]) / dd_a
# dd_k = np.pi * dd_n * (dd_n + 1) * (dd_n + 2) * dd_a
# np.round(dd_k / 2)

array([[   7.,    7.,    7.,    7.,    7.,    7.,    7.,    7.,    7.,
           7.,    7.],
       [  13.,   13.,   13.,   13.,   13.,   13.,   13.,   13.,   13.,
          13.,   13.],
       [  16.,   16.,   16.,   16.,   16.,   16.,   16.,   16.,   16.,
          16.,   16.],
       [  25.,   25.,   25.,   25.,   25.,   25.,   25.,   25.,   25.,
          25.,   25.],
       [  34.,   34.,   34.,   34.,   34.,   34.,   34.,   34.,   34.,
          34.,   34.],
       [  43.,   43.,   43.,   43.,   43.,   43.,   43.,   43.,   43.,
          43.,   43.],
       [  54.,   54.,   54.,   54.,   54.,   54.,   54.,   54.,   54.,
          54.,   54.],
       [  69.,   69.,   69.,   69.,   69.,   69.,   69.,   69.,   69.,
          69.,   69.],
       [  84.,   84.,   84.,   84.,   84.,   84.,   84.,   84.,   84.,
          84.,   84.],
       [  99.,   99.,   99.,   99.,   99.,   99.,   99.,   99.,   99.,
          99.,   99.],
       [ 114.,  114.,  114.,  114.,  114.,  114.,  114.,  11